In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df_train = pd.read_csv('train.csv') 

In [ ]:
train_path = 'train/'
X_train = []
y_train = []

for index, row in df_train.iterrows():
    article_id = int(row['id'])
    folder_name = f"article_{article_id:04d}"
    
    real_file_num = int(row['real_text_id'])
    
    for file_num in [1, 2]:
        file_path = f"{train_path}{folder_name}/file_{file_num}.txt"
        
        if os.path.exists(file_path):
            with open(file_path, encoding='utf-8') as file:
                X_train.append(file.read())
                
                if file_num == real_file_num:
                    y_train.append(1)
                else:
                    y_train.append(0)

In [ ]:
test_path = 'test/'
X_test = []

test_folders = sorted(os.listdir(test_path), key=lambda x: int(x.split('_')[1]))

for folder in test_folders:
    for file_num in [1, 2]:
        file_path = f"{test_path}{folder}/file_{file_num}.txt"
        if os.path.exists(file_path):
            with open(file_path, encoding='utf-8') as file:
                X_test.append(file.read())

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

tfidf = TfidfVectorizer(max_features=5000, stop_words='english', ngram_range=(1, 2))

X_train_tfidf = tfidf.fit_transform(X_train) 
X_test_tfidf = tfidf.transform(X_test)

rf = RandomForestClassifier(n_estimators=300, max_depth=20, random_state=42)
rf.fit(X_train_tfidf, y_train)

In [ ]:
test_probs = rf.predict_proba(X_test_tfidf)[:, 1]

In [ ]:
final_answers = []

for i in range(0, len(test_probs), 2):
    prob_file_1 = test_probs[i]
    prob_file_2 = test_probs[i+1]
    
    if prob_file_1 >= prob_file_2:
        final_answers.append(1)
    else:
        final_answers.append(2)

In [ ]:
submission_df = pd.DataFrame({
    'id': range(len(final_answers)),
    'real_text_id': final_answers
})

submission_df.to_csv('submission.csv', index=False)